<a href="https://colab.research.google.com/github/pin164/Automated-Expense-Categorization/blob/main/Decision_Tree_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# STEP 1
# Import  dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn import tree
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier


from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preprocessing

In [29]:
# STEP 2
#  Import and read the Cleaned_Invoice_4.csv
#/content/drive/MyDrive/Project 4/Automated-Expense-Categorization/Cleaned_Invoice_4.csv
application_df = pd.read_csv("/content/drive/MyDrive/Project 4/Automated-Expense-Categorization/Cleaned_Invoice_4.csv",encoding='latin1').iloc[:,:14]
application_df.head()

,Vendor,Month,Year,PO#,Invoice Date,Invoice#,total Invoice Amt,TRS,Project,Amount,Cost Centre,Order#,Dept
0,vendor 1039,Apr,2021,4501190248,2021-04-30,4,"$18,850.00",SDS7089109,WSH LINK Replacement - D&I,$455.00,165239,300007147,FIN
1,vendor 1039,Apr,2021,4501190248,2021-04-30,4,"$18,850.00",SDS7090144,AIMS Replacement (PDO-217),"$3,575.00",171318,930089792,BTT
2,vendor 1039,Apr,2021,4501190248,2021-04-30,4,"$18,850.00",SDS7089063,Petrinex - D&I,$877.50,190230,300007150,ARD
3,vendor 1039,Apr,2021,4501190248,2021-04-30,4,"$18,850.00",SDS7090248,M3P Vitality Upgrade - Implementation Phase - ...,"$4,062.50",195006,300007300,JUS
4,vendor 1039,Apr,2021,4501180577,2021-04-30,5,"$18,850.00",SDS7089423,Civil Legal Billing System - Scoping,"$5,005.00",171318,300007265,JUS


In [30]:
# STEP 3
# extract beneficial columns only
####X_df = application_df[['Column2','PO#','TRS','Order#','Column3']]
X_df = application_df[['PO#','TRS','Order#']]
Y_df = application_df['Cost Centre']
X_df.head()


,PO#,TRS,Order#
0,4501190248,SDS7089109,300007147
1,4501190248,SDS7090144,930089792
2,4501190248,SDS7089063,300007150
3,4501190248,SDS7090248,300007300
4,4501180577,SDS7089423,300007265


In [31]:
Y_df.value_counts()


,count
Cost Centre,
171318,2225
171293,869
171225,546
171194,394
165155,195
...,...
175259,5
171190,5
171197,4


In [32]:
X_df.dtypes

,0
PO#,object
TRS,object
Order#,int64


In [33]:
#Step 4
# amend the data type of order to be an object
X_df.loc[:, 'Order#'] = X_df['Order#'].astype('object')

X_df.dtypes

<ipython-input-33-78b2616e01c1>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[300007147 930089792 300007150 ... 300007069 300007069 300007069]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_df.loc[:, 'Order#'] = X_df['Order#'].astype('object')


,0
PO#,object
TRS,object
Order#,object


In [34]:
Y_df.head()

,Cost Centre
0,165239
1,171318
2,190230
3,195006
4,171318


In [35]:
#Step 5
application_df_encoded = pd.get_dummies(X_df)
application_df_encoded

,PO#_4501098288,PO#_4501098743,PO#_4501102419,PO#_4501105954,PO#_4501105971,PO#_4501107026,PO#_4501112913,PO#_4501118863,PO#_4501119241,PO#_4501119371,...,Order#_930092701,Order#_930092720,Order#_930092721,Order#_930092795,Order#_930092863,Order#_930092864,Order#_930092877,Order#_930092879,Order#_930093000,Order#_930093243
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7273,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7274,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7275,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7276,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [36]:
# Display the first few rows of the newly encoded dataframe
# This will show how the categorical columns have been transformed into numerical ones
print("\nEncoded DataFrame (First 5 Rows):")
print(application_df_encoded.head())


Encoded DataFrame (First 5 Rows):
   PO#_4501098288  PO#_4501098743  PO#_4501102419  PO#_4501105954  \
0           False           False           False           False   
1           False           False           False           False   
2           False           False           False           False   
3           False           False           False           False   
4           False           False           False           False   

   PO#_4501105971  PO#_4501107026  PO#_4501112913  PO#_4501118863  \
0           False           False           False           False   
1           False           False           False           False   
2           False           False           False           False   
3           False           False           False           False   
4           False           False           False           False   

   PO#_4501119241  PO#_4501119371  ...  Order#_930092701  Order#_930092720  \
0           False           False  ...             False 

In [37]:
#Check for Missing Values: Before proceeding,  check if dataset contains any missing values (NaNs).
# Check for missing values in the feature set
print(application_df_encoded.isnull().sum())

PO#_4501098288      0
PO#_4501098743      0
PO#_4501102419      0
PO#_4501105954      0
PO#_4501105971      0
                   ..
Order#_930092864    0
Order#_930092877    0
Order#_930092879    0
Order#_930093000    0
Order#_930093243    0
Length: 837, dtype: int64


In [ ]:
# Check for missing values in the target variable
print("number of misisng target values ",Y_df.isnull().sum())

number of misisng target values  0


In [38]:
#Drop Rows with Missing Target Values:
#remove the rows from both the feature set X and the target set Y_df that contain missing target values.
# Combine X and y to drop rows with missing target values
combined_df = pd.concat([application_df_encoded, Y_df], axis=1)

# Drop rows where the target variable is missing
combined_df.dropna(subset=['Cost Centre'], inplace=True)

# Separate the features and target again after dropping missing values
X = combined_df.drop(columns=['Cost Centre']).values
y = combined_df['Cost Centre'].values

# Re-split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting datasets to verify the split
print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)
print("Training Target Shape:", y_train.shape)
print("Testing Target Shape:", y_test.shape)


Training Features Shape: (5822, 837)
Testing Features Shape: (1456, 837)
Training Target Shape: (5822,)
Testing Target Shape: (1456,)


## Fitting the decision tree classifier instance

In [39]:
# Step 1: Import the DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Step 2: Initialize the Decision Tree Model
dt_model = DecisionTreeClassifier(random_state=42)
# Increase the depth of the tree or minimum number of samples required to split
dt_model = DecisionTreeClassifier(random_state=42, max_depth=10, min_samples_split=5)

# Step 3: Train the Model on the Training Data
dt_model.fit(X_train, y_train)

# Step 4: Make Predictions on the Test Data
y_pred = dt_model.predict(X_test)





In [40]:
# Check the distribution of target values
print(pd.Series(y_train).value_counts())


171318    1783
171293     695
171225     427
171194     315
165155     159
          ... 
175828       5
175259       5
171197       4
171190       3
195496       2
Name: count, Length: 72, dtype: int64


In [41]:
# Step 5: Evaluate the Model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("Classification Report:")
print(classification_report(y_test, y_pred, zero_division=1))

confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(confusion)


Accuracy: 0.4416
Classification Report:
              precision    recall  f1-score   support

      110508       1.00      0.00      0.00         8
      110889       1.00      0.00      0.00        13
      111173       1.00      0.00      0.00        15
      111508       1.00      0.00      0.00        21
      135072       1.00      0.00      0.00        12
      135139       1.00      0.00      0.00         5
      135150       1.00      0.00      0.00        13
      135367       1.00      0.00      0.00         7
      140010       1.00      1.00      1.00        15
      140548       1.00      0.00      0.00        19
      140642       1.00      0.00      0.00         4
      160207       1.00      0.00      0.00         2
      160781       1.00      0.00      0.00         3
      160996       1.00      0.00      0.00         4
      161017       1.00      0.00      0.00        10
      163124       1.00      0.00      0.00         5
      165031       1.00      0.00      0.

In [42]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
rf_model = RandomForestClassifier(random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions and evaluate
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=1))

              precision    recall  f1-score   support

      110508       1.00      1.00      1.00         8
      110889       1.00      1.00      1.00        13
      111173       1.00      1.00      1.00        15
      111508       1.00      1.00      1.00        21
      135072       1.00      1.00      1.00        12
      135139       1.00      1.00      1.00         5
      135150       1.00      1.00      1.00        13
      135367       1.00      1.00      1.00         7
      140010       1.00      1.00      1.00        15
      140548       1.00      1.00      1.00        19
      140642       1.00      1.00      1.00         4
      160207       1.00      1.00      1.00         2
      160781       1.00      1.00      1.00         3
      160996       1.00      1.00      1.00         4
      161017       1.00      1.00      1.00        10
      163124       1.00      1.00      1.00         5
      165031       1.00      1.00      1.00         9
      165039       1.00    

In [43]:
rf_confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(rf_confusion)

Confusion Matrix:
[[ 8  0  0 ...  0  0  0]
 [ 0 13  0 ...  0  0  0]
 [ 0  0 15 ...  0  0  0]
 ...
 [ 0  0  0 ...  7  0  0]
 [ 0  0  0 ...  0 16  0]
 [ 0  0  0 ...  0  0  9]]
